In [13]:
!pip install -qU langchain langsmith langchain-core langchain-community langchain-experimental langchain-openai langchain-groq pypdf langchain-text-splitters langchain-chroma faiss-cpu sentence_transformers google-search-results wikipedia duckduckgo-search arxiv langgraph

In [14]:
import os
from google.colab import userdata
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')

In [15]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGSMITH_API_KEY')
os.environ["LANGCHAIN_PROJECT"] = "langgraph_learning"

In [16]:
from langchain_community.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun() # Web Search


from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper()) # Wikipedia search

In [17]:
import json

def extact_wiki_result(result):

  # Find the start of the first summary
  start = result.find("Summary:")  # Find the first occurrence of 'Summary:'

  # Find the end of the first summary (next 'Page:' or the end of the string)
  end = result.find("Page:", start)  # Find the next 'Page:' after the first summary

  if end == -1:  # If there is no subsequent 'Page:', take the entire remaining text
      end = len(result)

  # Extract the first summary
  first_summary = result[start+len("Summary:"):end].strip()

  # print(first_summary)

  return first_summary

In [18]:
def duckduckgo(query):
  result = search.invoke(query)
  return result

def wiki_pedia(query):
  result = wikipedia.invoke(query)
  return extact_wiki_result(result)

In [19]:
known_actions = {
    "wiki_pedia": wiki_pedia,
    "duckduckgo": duckduckgo
}

In [20]:
prompt = """
  You run in a loop of Thought, Action, PAUSE, Observation.
  At the end of the loop your output an Answer.

  Use Thought to describe your thoughts about the question you have been asked.
  Use Action to run one of the actions available to you - then return PAUSE.
  Observation will be the result of running those actions.


  Your available actions are:
  wiki_pedia:
  e.g. wiki_pedia: Django
  Returns a summary from searching Wikipedia

  duckduckgo:
  e.g. duckduckgo: Python
  Return search summary about python

  Example session:
  Question: What is the capital of France?
  Thought: I should look up France on Wikipedia
  Action: duckduckgo: France
  PAUSE

  You will be called again with this:
  Observation: France is a country. The capital is Paris.

  You then output:
  Answer: The capital of France is Paris

  Please Note:
  If you get basic conversation questions like "hi","hello","how are you?",\n
  you have to answer "hi","hello","i am good".

""".strip()

In [21]:
from langchain_groq import ChatGroq
import re

# Compiling the regular expression pattern
action_re = re.compile('^Action: (\w+): (.*)') # Extract the action and observation from the prompt


In [22]:
class ChatBot:
  def __init__(self, system = ""):
    self.system = system # System behavior
    self.message = []

    if self.system:
      self.message.append({"role":"system","content":system})

  def __call__(self, message):
    self.message.append({"role": "user", "content": message}) # append user message

    result = self.execute() # Run LLM

    self.message.append({"role": "assistant", "content": result}) # append assistant message
    return result

  def execute(self):
    llm = ChatGroq(model = 'llama-3.1-8b-instant')
    result = llm.invoke(self.message)
    return result.content

In [23]:
def user_query(query, max_iteration = 5):
  count = 0 # Initialize counter for iteration

  bot = ChatBot(prompt) # System behavior is now having the prompt instruction

  next_prompt = query # This will keep getting replaced with observation

  while count < max_iteration:
    # Call the Agent
    result = bot(next_prompt)
    print("result: ", result)

    actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)] # Grab the action from the string
    print(actions)

    if actions:
      """Further action is needed to find the final answer"""
      action , action_input = actions[0].group(1), actions[0].group(2)

      if action not in known_actions:
        # No such tool is present
        raise Exception(f"Unknown action: {action}: {action_input}")

      print(" -- running {} {}".format(action, action_input))

      # Use the tool to find the anser
      observation = known_actions[action](action_input)

      print("Observation: ", observation)

      next_prompt = f"Observation: {observation}"

      print("next prompt", next_prompt)

      # Increase the counter
      count += 1

    else:
      """No action was needed, found the final result"""
      return result


In [24]:
if __name__ == '__main__':
  user_input = input("ask: ")
  if user_input == "q":
    print("Bye")
  else:
    result = user_query(user_input)
    print(result)

ask: What is the current temperature in Michigan?
result:  Thought: I should look up the current temperature in Michigan online, possibly using a search engine to find the latest weather information.

Action: duckduckgo: Michigan temperature
PAUSE
[<re.Match object; span=(0, 40), match='Action: duckduckgo: Michigan temperature'>]
 -- running duckduckgo Michigan temperature
Observation:  Today, in Michigan, a combination of snowy and cloudy weather is predicted. With minimal precipitation of 0.04" (1mm), snowfall is anticipated from 11 am to 4 pm and at 8 pm. The temperature will be between the lowest temperature of 24.8°F (-4°C) and the highest temperature of 32°F (0°C). For More Weather Information: Grand Rapids, MI Local Forecast Office. 24NM W Ludington MI Marine Point Forecast. This Afternoon. High: 30 °F. WNW 17kt 3ft. Tonight. Low: 26 °F. NW 22kt 3-5ft. Friday. High: 28 °F. NW 23kt 5-6ft. Friday Night. Marquette, MI Severe Weather and Heavy Rainfall Potential across the East; Atm

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Observation:  Wind chill (popularly wind chill factor) is the sensation of cold produced by the wind for a given ambient air temperature on exposed skin as the air motion accelerates the rate of heat transfer from the body to the surrounding atmosphere. Its values are always lower than the air temperature in the range where the formula is valid. When the apparent temperature is higher than the air temperature, the heat index is used instead.
next prompt Observation: Wind chill (popularly wind chill factor) is the sensation of cold produced by the wind for a given ambient air temperature on exposed skin as the air motion accelerates the rate of heat transfer from the body to the surrounding atmosphere. Its values are always lower than the air temperature in the range where the formula is valid. When the apparent temperature is higher than the air temperature, the heat index is used instead.
result:  Thought: I'm familiar with the concept of wind chill, but the provided information doesn